## Multi Agentic Job Application Helper with CrewAI and OpenAI GPT-4

In [ ]:
# pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from job_posting_details import *

In [ ]:
job_posting_url

In [ ]:
company_name 

In [ ]:
job_role_name

In [ ]:
print(job_description)

In [ ]:
destination_folder = company_name + '/' + job_role_name
destination_folder

In [ ]:
# save job description
import os
filename = destination_folder + '/job_description.txt'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, "w") as f:
    f.write(job_description)

### CREW AI AGENTS AND TASK

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
# AGENTS

In [ ]:
import os
# from utils import get_openai_api_key, get_serper_api_key


os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = "<SERPER_API_KEY>"
# sign up for serper api key here https://serper.dev/

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./jonathan_musni_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./jonathan_musni_resume.md')

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./jonathan_musni_resume.md"))

#### Agents

In [ ]:
from agent_parameters import *

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role = "Tech Job Researcher",
    goal = researcher_agent_goal,
    verbose = True,
    backstory = (researcher_agent_backstory)
)

# Agent 2: Profiler
profiler = Agent(
    role = "Personal Profiler for Engineers",
    goal = profiler_agent_goal,
#     tools = [scrape_tool, search_tool,
#              read_resume, semantic_search_resume],
#     tools = [read_resume, semantic_search_resume],
    tools = [read_resume],

    verbose = True,
    backstory = (profiler_agent_backstory)
)

# Agent 3: Resume Strategist
resume_strategist = Agent(
    role = "Resume Strategist for Engineers",
    goal = resume_strategist_agent_goal,
#     tools = [read_resume, semantic_search_resume],
    tools = [read_resume],

    verbose = True,
    backstory = (resume_strategist_agent_backstory)
)

# Agent 4: Quality Checker
quality_checker = Agent(
    role = "Resume Honesty Checker",
    goal = quality_checker_agent_goal,
#     tools = [read_resume, semantic_search_resume],
    tools = [read_resume],
    verbose = True,
    backstory = (quality_checker_agent_backstory)
)

# Agent 5: Cover Letter Generator
cover_letter_generator = Agent(
    role = "Cover Letter Generator",
    goal = cover_letter_generator_goal,
#     tools = [read_resume, semantic_search_resume],
    tools = [read_resume],
    verbose = True,
    backstory = (cover_letter_generator_backstory)
)

# Agent 6: Interview Preparer
interview_preparer = Agent(
    role = "Engineering Interview Preparer",
    goal = interview_preparer_agent_goal,
#     tools = [scrape_tool, search_tool,
#              read_resume, semantic_search_resume],
    tools = [scrape_tool, search_tool,
             read_resume],
    verbose = True,
    backstory = (interview_preparer_agent_backstory)
)

#### TASKS

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description = (research_task_description),
    expected_output = (research_task_expected_output),
    agent = researcher,
    async_execution = True
)

# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description = (profiler_task_description),
    expected_output = (profiler_task_expected_output),
    agent = profiler,
    async_execution = True
)

# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description = (resume_strategy_task_description),
    expected_output = (resume_strategy_task_expected_output),
    output_file = destination_folder + "/tailored_resume_draft.md",
    context = [research_task, profile_task],
    agent = resume_strategist
)

# Task for Quality Check Agent: Check for honesty and remove exaggerations
quality_check_task = Task(
    description = (quality_check_task_description),
    expected_output = (quality_check_task_expected_output),
    output_file = destination_folder + "/tailored_resume_final.md",
    context = [research_task, profile_task,resume_strategy_task],
    agent = quality_checker
)

# Task for Cover Letter Agent: Generate a cover letter
cover_letter_task = Task(
    description = (cover_letter_task_description),
    expected_output = (cover_letter_task_expected_output),
    output_file = destination_folder + "/cover_letter.md",
    context = [research_task, profile_task, resume_strategy_task, quality_check_task],
    agent = cover_letter_generator
)

# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description = (interview_preparation_task_description),
    expected_output = (interview_preparation_task_expected_output),
    output_file = destination_folder + "/interview_materials.md",
    context = [research_task, profile_task, resume_strategy_task, quality_check_task],
    agent = interview_preparer
)


In [ ]:
job_application_crew = Crew(
    agents = [researcher,
            profiler,
            resume_strategist,
            quality_checker,
            cover_letter_generator,
            interview_preparer],

    tasks = [research_task,
           profile_task,
           resume_strategy_task,
           quality_check_task,
#            revision_details_task,
           interview_preparation_task,
           cover_letter_task],

    verbose=True
)

In [ ]:
job_application_inputs = {
    'job_description': job_description,
    'cover_letter_inputs' : cover_letter_inputs,
    'personal_writeup': personal_writeup
}

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

In [ ]:
from IPython.display import Markdown, display
display(Markdown(destination_folder + "/tailored_resume_draft.md"))

In [ ]:
from IPython.display import Markdown, display
display(Markdown(destination_folder + "/tailored_resume_final.md"))